In [1]:
import numpy as np

**Функция, возвращающая размерность выхода**

In [2]:
def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    input_matrix_shape[2] = (input_matrix_shape[2] + 2*padding - (kernel_size - 1) - 1) // stride + 1
    input_matrix_shape[3] = (input_matrix_shape[3] + 2*padding - (kernel_size - 1) - 1) // stride + 1
    input_matrix_shape[1] = out_channels
    return input_matrix_shape

In [3]:
calc_out_shape(input_matrix_shape=[2, 3, 10, 10],
               out_channels=10,
               kernel_size=3,
               stride=1,
               padding=0)

[2, 10, 8, 8]

**Функцию, добавляющая padding**

In [4]:
def get_padding2d(input_images):
    padded_images = torch.zeros([2, 3, 5, 5], dtype=torch.float32)
    padded_images[:, :, 1:-1, 1:-1] += input_images[:, :, :, :].type(torch.FloatTensor)
    return padded_images

**Реализации сверточного слоя через циклы**

In [5]:
import torch
from abc import ABC, abstractmethod


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)


# Сверточный слой через циклы.
class Conv2dLoop(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        
    def set_kernel(self, kernel):
        self.kernel = kernel
    
    def __call__(self, input_tensor):
        
        input_height = input_tensor.shape[2]
        input_width = input_tensor.shape[3]
        batch_size = input_tensor.shape[0]
        
        output_height = (input_height - (self.kernel_size - 1) - 1) // self.stride + 1
        output_width = (input_width - (self.kernel_size - 1) - 1) // self.stride + 1

        output_tensor = torch.zeros(batch_size * self.out_channels * self.in_channels * output_height * output_width, out=torch.FloatTensor())
        k = 0

        for batch in range(batch_size):
            for out_channel in range(self.out_channels):
                for in_channel in range(self.in_channels):
                    tens = input_tensor[batch][in_channel]
                    kern = self.kernel[out_channel][in_channel]
                    for i in range(0, input_height, self.stride):
                        for j in range(0, input_width, self.stride):
                            candidat = tens[i:self.kernel_size+i, j:self.kernel_size+j]
                            if candidat.shape[0] == self.kernel_size and candidat.shape[1] == self.kernel_size:
                                point = (candidat * kern).sum().item()
                                output_tensor[k] = point
                                k += 1
        output_tensor = output_tensor.reshape(batch_size, self.in_channels, output_height, output_width).sum(dim=1, keepdim=True)        

        return output_tensor

In [6]:
print(test_conv2d_layer(Conv2dLoop))

True
